In [2]:
import torch
import os
import torchaudio
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F

In [3]:
from utils.dataloader import make_sorted_dataloader, collate_with_paddings
from utils.chunk_dataset import ChunkDataSet
from utils.zip_dataset import ZippedDataSet
from utils.chunk_augmented_dataset import ChunkAugDataSet

In [4]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2Model, Wav2Vec2Processor

In [17]:
#tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-base-960h')
#model = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')
class Wav2Vec2ED(nn.Module): 
    def __init__(self,  sample_rate=16000, n_classes=41):
        super().__init__()
        #self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)
        #self.bn0 = nn.BatchNorm2d(1)
        
        #self.cnn1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)
        #self.bn1 = nn.BatchNorm2d(16)
        #self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        #self.bn2 = nn.BatchNorm2d(32)
        #self.cnn3 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=3, padding=1)
        #self.bn3 = nn.BatchNorm2d(3)
        self.dropout=nn.Dropout(0.1)
        
        self.features = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        for param in  self.features.encoder.pos_conv_embed.parameters():
            param.requires_grad = False
        # use it as features
#         for param in self.features.parameters():
#             param.requires_grad = False
        #self.mp = torch.nn.MaxPool1d(3)
        self.lin1 = nn.Linear(768, 111)
              
        self.lin2 = nn.Linear(111, n_classes)
        self.freeze = True

    def forward(self, x):
        #x = self.ms(x)
        #x = self.bn0(x)
        #x = F.relu(self.cnn1(x))
        #x = self.dropout(F.relu(self.bn1(self.cnn1(x))))
        #x = self.dropout(F.relu(self.bn2(self.cnn2(x))))
        #x = F.relu(self.bn3(self.cnn3(x)))
        if self.freeze:
            with torch.no_grad():
                x = self.features(x).last_hidden_state
        else:
            x = self.features(x).last_hidden_state

#         x = x.view(x.shape[0], -1)
        #print(x.shape)
        # x = F.relu(x.transpose(1, 2))
        # x = self.mp(x)
        # x = F.relu(self.conv1d1(x))
        # x = F.relu(self.conv1d2(x))
        # #print(x.shape)
        # x = x.flatten(1)
        #x = torch.mean(x, dim=1)
        #x = x.flatten(1)
        #x = F.relu(x)
        
        x = F.relu(self.lin1(x[:,-1,:]))
        #x = F.relu(self.lin2(x))
        x = self.lin2(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x, dim=-1)
        return x

out = Wav2Vec2ED()(torch.ones(1, 32000))
print(out.shape)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.weight', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 41])


In [18]:
#train_ds = 
# chunk_sz=6
# train_ds = ZippedDataSet(f'data/Annotations_gold_standard/English/train_chunk{chunk_sz}s_msl0.zip', ChunkDataSet('./data/Annotations_gold_standard/English/train/', chunk_size_s=chunk_sz), rewrite=False, min_sample_len=0)
# test_ds = ZippedDataSet(f'data/Annotations_gold_standard/English/test_chunk{chunk_sz}s_msl0.zip', ChunkDataSet('./data/Annotations_gold_standard/English/test', chunk_size_s=chunk_sz), rewrite=False, min_sample_len=0)

In [19]:
wav, sr = torchaudio.load('data/Annotations_gold_standard/English/train/027_2016-04-06_Nottingham/expert.audio[48000]_clean.wav')
wav = torchaudio.functional.resample(wav, sr, 16000)

In [20]:
model = Wav2Vec2ED().cuda()

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.weight', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
out = model(wav[:, :5000].cuda())

In [22]:
out.shape

torch.Size([1, 41])

In [23]:
out

tensor([[ 0.0621,  0.0606, -0.0752,  0.0157, -0.0388, -0.0562,  0.1037,  0.1108,
         -0.1186,  0.1355, -0.0447,  0.1238,  0.0658, -0.0163, -0.0455,  0.1989,
          0.0986, -0.0378,  0.0057,  0.0622,  0.0840,  0.0244,  0.0667,  0.0345,
         -0.0471,  0.1396, -0.0814,  0.1465, -0.0858,  0.0294, -0.0677,  0.0872,
          0.1620, -0.0876,  0.0252, -0.0770, -0.0569,  0.0558,  0.0677, -0.1311,
          0.1163]], device='cuda:0', grad_fn=<AddmmBackward>)